In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from graphtorch import SparseMatrix, SparseModel
import numpy as np
import torch 
import torch.nn as nn

__Example 1__

In [3]:
mat1 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])  
in_dim = 5   
out_dim = 2  
mat_wann1 = SparseMatrix(mat1, in_dim, out_dim)  

In [4]:
activations = [None, None, nn.ReLU(), nn.Sigmoid()]  
constant_weight = 1 
model = SparseModel(mat_wann1, activations, constant_weight)

numpy_input = np.array([[1,2,3,4,5],  
                        [6,7,8,9,10],  
                        [11,12,13,14,15]])      

numpy_input = torch.from_numpy(numpy_input).float()  
output, nodes = model(numpy_input)  

In [5]:
output

tensor([[1.0000, 1.0000],
        [1.0000, 1.0000],
        [1.0000, 1.0000]], grad_fn=<CatBackward>)

In [6]:
nodes

{'hidden_0': tensor([[ 7.],
         [17.],
         [27.]], grad_fn=<AddBackward0>), 'hidden_1': tensor([[ 4.],
         [14.],
         [24.]], grad_fn=<AddBackward0>), 'hidden_2': tensor([[ 6.],
         [16.],
         [26.]], grad_fn=<AddBackward0>), 'hidden_3': tensor([[11.],
         [31.],
         [51.]], grad_fn=<AddBackward0>), 'hidden_4': tensor([[10.],
         [30.],
         [50.]], grad_fn=<AddBackward0>), 'output_0': tensor([[1.0000],
         [1.0000],
         [1.0000]], grad_fn=<SigmoidBackward>), 'output_1': tensor([[1.0000],
         [1.0000],
         [1.0000]], grad_fn=<SigmoidBackward>)}

__Example 2__

In [7]:
mat2 = np.array([[2,0,0,0,0,0],
                [0,0,0,3,0,1]])
in_dim = 5
out_dim = 1
mat_wann2 = SparseMatrix(mat2, in_dim, out_dim)  

In [8]:
activations = [None, None, nn.ReLU(), nn.Sigmoid()]  
constant_weight = 1 
model = SparseModel(mat_wann2, activations, constant_weight)

numpy_input = np.array([[1,2,3,4,5],  
                        [6,7,8,9,10],  
                        [11,12,13,14,15]]) 

numpy_input = torch.from_numpy(numpy_input).float()  
output, nodes = model(numpy_input)  

In [9]:
output

tensor([[ 1.9820],
        [ 6.9999],
        [12.0000]], grad_fn=<AddBackward0>)

In [10]:
nodes

{'hidden_0': tensor([[ 1.],
         [ 6.],
         [11.]], grad_fn=<ReluBackward0>), 'output_0': tensor([[ 1.9820],
         [ 6.9999],
         [12.0000]], grad_fn=<AddBackward0>)}

__Example 3__

In [11]:
mat3 = np.array([[2,0,0,3,0],
                [0,0,0,0,1]])    
in_dim = 5  
out_dim = 2    
mat_wann3 = SparseMatrix(mat3, in_dim, out_dim)

In [12]:
activations = [None, None, nn.ReLU(), nn.Sigmoid()]  
constant_weight = 1 
model = SparseModel(mat_wann3, activations, constant_weight)

numpy_input = np.array([[1,2,3,4,5],  
                        [6,7,8,9,10],  
                        [11,12,13,14,15]]) 

numpy_input = torch.from_numpy(numpy_input).float()  
output, nodes = model(numpy_input)  

In [13]:
output

tensor([[ 1.9820,  5.0000],
        [ 6.9999, 10.0000],
        [12.0000, 15.0000]], grad_fn=<CatBackward>)

In [14]:
nodes

{'output_0': tensor([[ 1.9820],
         [ 6.9999],
         [12.0000]], grad_fn=<AddBackward0>), 'output_1': tensor([[ 5.],
         [10.],
         [15.]], grad_fn=<MmBackward>)}

__Example 4__

In [15]:
mat4 = np.array([[3,0,1,0,0],
                [0,3,0,0,0],
                [0,0,0,2,0],
                [0,0,0,0,2],
                [0,0,1,2,0],
                [0,0,1,0,0]])
in_dim = 3
out_dim = 4
mat_wann4 = SparseMatrix(mat4, in_dim, out_dim)

In [16]:
activations = [None, None, nn.ReLU(), nn.Sigmoid()]  
constant_weight = 1 
model = SparseModel(mat_wann4, activations, constant_weight)

numpy_input = np.array([[1,2,3,4,5],  
                        [6,7,8,9,10],  
                        [11,12,13,14,15]]) 

numpy_input = torch.from_numpy(numpy_input).float()  
output, nodes = model(numpy_input)  

In [17]:
output

tensor([[ 3.7311,  0.8808,  6.7311,  3.0000],
        [ 8.9975,  0.9991, 16.9975,  8.0000],
        [14.0000,  1.0000, 27.0000, 13.0000]], grad_fn=<CatBackward>)

In [18]:
nodes

{'hidden_0': tensor([[ 3.7311],
         [ 8.9975],
         [14.0000]], grad_fn=<AddBackward0>), 'hidden_1': tensor([[0.8808],
         [0.9991],
         [1.0000]], grad_fn=<SigmoidBackward>), 'output_0': tensor([[ 3.7311],
         [ 8.9975],
         [14.0000]], grad_fn=<ReluBackward0>), 'output_1': tensor([[0.8808],
         [0.9991],
         [1.0000]], grad_fn=<ReluBackward0>), 'output_2': tensor([[ 6.7311],
         [16.9975],
         [27.0000]], grad_fn=<AddBackward0>), 'output_3': tensor([[ 3.],
         [ 8.],
         [13.]], grad_fn=<MmBackward>)}

__Example 5__ 

In [19]:
mat5 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,2,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])
in_dim = 5
out_dim = 2
mat_wann5 = SparseMatrix(mat5, in_dim, out_dim)

In [20]:
activations = [None, None, nn.ReLU(), nn.Sigmoid()]  
constant_weight = 1 
model = SparseModel(mat_wann5, activations, constant_weight)

numpy_input = np.array([[1,2,3,4,5],  
                        [6,7,8,9,10],  
                        [11,12,13,14,15]]) 

numpy_input = torch.from_numpy(numpy_input).float()  
output, nodes = model(numpy_input)  

In [21]:
output

tensor([[1.0000, 1.0000],
        [1.0000, 1.0000],
        [1.0000, 1.0000]], grad_fn=<CatBackward>)

In [22]:
nodes

{'hidden_0': tensor([[ 7.],
         [17.],
         [27.]], grad_fn=<AddBackward0>), 'hidden_1': tensor([[ 4.],
         [14.],
         [24.]], grad_fn=<AddBackward0>), 'hidden_2': tensor([[ 6.],
         [16.],
         [26.]], grad_fn=<AddBackward0>), 'hidden_3': tensor([[11.],
         [31.],
         [51.]], grad_fn=<AddBackward0>), 'hidden_4': tensor([[14.],
         [39.],
         [64.]], grad_fn=<AddBackward0>), 'output_0': tensor([[1.0000],
         [1.0000],
         [1.0000]], grad_fn=<SigmoidBackward>), 'output_1': tensor([[1.0000],
         [1.0000],
         [1.0000]], grad_fn=<SigmoidBackward>)}